In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import sqlite3
from datetime import datetime

In [2]:
resto = ["https://www.crous-strasbourg.fr/restaurant/cafeteria-le-pege-2/",
         "https://www.crous-strasbourg.fr/restaurant/resto-u-gallia-2/",
         "https://www.crous-strasbourg.fr/restaurant/resto-u-esplanade-2/",
         "https://www.crous-strasbourg.fr/restaurant/resto-u-paul-appell/",
         "https://www.crous-strasbourg.fr/restaurant/le-32-2/",
         "https://www.crous-strasbourg.fr/restaurant/lannexe-2/",
         "https://www.crous-strasbourg.fr/restaurant/resto-u-illkirch-2/",
         "https://www.crous-strasbourg.fr/restaurant/cafeteria-mini-r-2/",
         "https://www.crous-strasbourg.fr/restaurant/resto-u-cronenbourg-2/",
         "https://www.crous-strasbourg.fr/restaurant/le-cristal-shop-ru-esplanade-2/"
        ]

#verificare cosa offrono
cafe = ["https://www.crous-strasbourg.fr/restaurant/cafeteria-de-la-fac-de-droit-2/",
        "https://www.crous-strasbourg.fr/restaurant/cafeteria-le-cardo-2/",
        "https://www.crous-strasbourg.fr/restaurant/le-stift/",
        "https://www.crous-strasbourg.fr/restaurant/cafeteria-la-misha-2/", #indirizzo: All. du Général Rouvillois, 67000 Strasbourg
        "https://www.crous-strasbourg.fr/restaurant/cafeteria-le-patio-2/",
        "https://www.crous-strasbourg.fr/restaurant/cafeteria-mini-r-2/"
]

# Function scrape information restaurant

In [3]:
def ScrapeInfo():
    restoInfo = {"name" : [], "link" : [], "hours": [], "address": [], "payment": [], "services": []}
    
    driver = webdriver.Firefox()
    
    for i in resto:
        driver.get(i)

        element = WebDriverWait(driver, 10).until(
            EC.text_to_be_present_in_element((By.CSS_SELECTOR, 'div.info'), "HORAIRES")
        )

        try:
            reject_all = driver.find_element(By.XPATH, "/html/body/div[6]/div/div[2]/div/div[2]/button[1]")
            reject_all.click()
        except Exception:
            try:
                reject_all = driver.find_element(By.ID, "tru_deselect_btn")
                reject_all.click()
            except Exception:
                pass

            
        restoInfo["name"].append(re.search(r'/([a-zA-Z0-9\-]+?)(?:-\d+)?/$', i).group(1).replace("-"," "))
            
        restoInfo['link'].append(i)

        info_div = driver.find_elements(By.CSS_SELECTOR, 'div.info')
        for div in info_div:
            
            if "HORAIRES" in div.text:
                lines = div.text.split('\n')
                restoInfo["hours"].append(" ".join(lines[1:]))

            if "ADRESSE" in div.text:
                lines = div.text.split('\n')
                address = lines[1:][0]
                if "67000" not in address and "67400" not in address and "67200" not in address:
                    address = address + " 67000 Strasbourg"
                restoInfo["address"].append(address)        

            if "PAIEMENT POSSIBLE" in div.text:
                lines = div.text.split('\n')
                restoInfo["payment"].append(", ".join(lines[1:]))

            if "PRATIQUE" in div.text:
                lines = div.text.split('\n')
                restoInfo["services"].append(", ".join(lines[1:]))

    driver.quit()

    return pd.DataFrame(restoInfo)
    
    

In [4]:
infoDB = ScrapeInfo()
infoDB

,name,link,hours,address,payment,services
0,cafeteria le pege,https://www.crous-strasbourg.fr/restaurant/caf...,07h30 à 14h30,61 avenue de la Forêt Noire - 67000 Strasbourg,"Carte bancaire, IZLY","Accès handicapé, Accès wifi"
1,resto u gallia,https://www.crous-strasbourg.fr/restaurant/res...,Ouvert du lundi au vendredi > Self et Brasser...,1 place de l'Université 67000 Strasbourg,"Carte bancaire, IZLY","Accès handicapé, Accès wifi"
2,resto u esplanade,https://www.crous-strasbourg.fr/restaurant/res...,Ouvert du lundi au vendredi > Self : 11h30 à ...,32 Boulevard de la Victoire 67000 Strasbourg,IZLY,"Accès handicapé, Accès wifi"
3,resto u paul appell,https://www.crous-strasbourg.fr/restaurant/res...,LE SELF vous accueille du lundi au vendredi de...,23 rue du Jura 67000 Strasbourg,IZLY,"Accès handicapé, Accès wifi"
4,le 32,https://www.crous-strasbourg.fr/restaurant/le-...,11h30 à 13h30,32 boulevard de la Victoire 67000 Strasbourg,"Carte bancaire, IZLY","Accès handicapé, Accès wifi"
5,lannexe,https://www.crous-strasbourg.fr/restaurant/lan...,"Du lundi au vendredi, de 11h30 à 13h45.",23 rue du Jura 67000 Strasbourg,IZLY,"Accès handicapé, Accès wifi"
6,resto u illkirch,https://www.crous-strasbourg.fr/restaurant/res...,Du lundi au vendredi : LE SELF est ouvert de ...,76 Route du Rhin 67400 Illkirch-Graffenstaden,IZLY,"Accès handicapé, Accès wifi"
7,cafeteria mini r,https://www.crous-strasbourg.fr/restaurant/caf...,De 11h30 à 13h45.,23 rue du Loess 67200 Strasbourg,IZLY,
8,resto u cronenbourg,https://www.crous-strasbourg.fr/restaurant/res...,Du lundi au vendredi : LE SELF est ouvert de ...,23 rue du Loess 67200 Strasbourg,"Carte bancaire, IZLY","Accès handicapé, Accès wifi"
9,le cristal shop ru esplanade,https://www.crous-strasbourg.fr/restaurant/le-...,> 7h30 à 14h15 Vente à emporter et à consomme...,32 boulevard de la Victoire 67000 Strasbourg,"Carte bancaire, IZLY",Accès wifi


In [5]:
conn = sqlite3.connect('info.db')
infoDB.to_sql('my_table', conn, if_exists='replace', index=False)
conn.close()

# Function scrape menu 

In [6]:
def get_today_date():
    # get today date
    today = datetime.today()
    # format the day like "jeudi 10 octobre"
    formatted_date = today.strftime("%A %d %B")
    return formatted_date

def ScrapeMenu(link, day):
    menuInfo = {"day": [], "lunch": [], "dinner": []}
    
    driver = webdriver.Firefox()
    driver.get(link)
    
    # Cookies button
    try:
        reject_all = driver.find_element(By.XPATH, "/html/body/div[6]/div/div[2]/div/div[2]/button[1]")
        reject_all.click()
    except Exception:
        try:
            reject_all = driver.find_element(By.ID, "tru_deselect_btn")
            reject_all.click()
        except Exception:
            pass
    
    # Find the right day
    date_elements = driver.find_elements(By.CSS_SELECTOR, 'time.menu_date_title')
    scraped_day = ' '.join(date_elements[0].text.split()[2:5])
    # Move until the right day
    while scraped_day == day:
        try:
            next_button = driver.find_element(By.XPATH, f"/html/body/article/div/section[1]/div/div/div[{j+1}]/div[1]/div[2]")
            next_button.click()
        except Exception as e:
            pass
    
    # Scrape meal
    meal_div = driver.find_elements(By.CSS_SELECTOR, 'div.meal')
    meals = []
    for meal in meal_div:
        if meal.text != "":
            meals.append(meal.text)
    
    if len(meals) == 2:
        lunch = meals[0].replace('\n', ' ')  # Ensure correct format for lunch
        dinner = meals[1].replace('\n', ' ')  # Ensure correct format for dinner
    elif len(meals) == 1:
        lunch = meals[0].replace('\n', ' ')
        dinner = ""
    else:
        lunch = ""
        dinner = ""
    
    # Do dataframe
    menuInfo["day"].append(day)
    menuInfo["lunch"].append(lunch)
    menuInfo["dinner"].append(dinner)

    driver.quit()
    
    return pd.DataFrame(menuInfo)

In [7]:
link = "https://www.crous-strasbourg.fr/restaurant/cafeteria-le-pege-2/"
day = get_today_date()

menuDB = ScrapeMenu(link, day)
menuDB

,day,lunch,dinner
0,Wednesday 16 October,Déjeuner Petit-déjeuner Café & Viennoiseries 7...,


In [9]:
conn = sqlite3.connect('menu.db')
menuDB.to_sql('my_table', conn, if_exists='replace', index=False)
conn.close()